# Reproduce DeepSeek R1 "Aha moment"

**DeepSeek-R1** is an open model that rivals OpenAI's o1 in complex reasoning tasks, introduced using **Group Relative Policy Optimization (GRPO)** and RL-forcused multi-stage training approach.

In this session, we will train an open model using RL trying to teach it self-verification and search abilities on its own to solve the Countdown Game, which is a number puzzle where players use a set of randomly drawn numbers and basic arithmetic operations (+, -, x, /) to reach or get as close as possible to a target number:
```text
Target Number: 952
Available Numbers: 25, 50, 75, 100, 3, 6

(100 x (3 x 3)) + (50 + 6 / 3) = 952
```

## Group Relative Policy Optimization (GRPO)

**Group Relative Policy Optimization (GRPO)** is a reinforcement learning algorithm to improve the reasoning capabilities of LLMs, which is introduced in the [DeepSeekMath](https://arxiv.org/abs/2402.03300) paper.

GRPO modifies the traditional **Proximal Policy Optimization (PPO)** by eliminating the need of a value function model. Instead, it estimates baselines from group scores, reducing memory usage and computational overhead.

GRPO, now also used by the Qwen team, can be used with rule/binary-based Rewards Models as well as General Rewards Models to improve models on helpfulness.

Steps of GRPO:
1. **Sampling** - Generating multiple outputs for each prompt using the current policy
2. **Reward Scoring** - Each generation is scored using a reward function(, could be rule-based or outcome-based)
3. **Advantage Calculation** - The average reward of the generated outputs is used as a baseline. The advantage of each solution within the group is them computed relative to this baseline. The reward is normalized within a group.
4. **Policy Optimization** - The policy tries to maximize the GRPO objective, which includes the calculated advantages and a KL divergence term. This is different from how PPO implements the KL term within the reward.

## Set up development environment

In [ ]:
!pip install "torch==2.5.1" tensorboard "setuptools<71.0.0"  --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install -qU flash-attn transformers datasets accelerate hf-transfer deepspeed trl vllm peft bitsandbytes

## Generate training samples with reasoning prefix from the Coundown Game

Dataset: [`Jiayi-Pan/Countdown-Tasks-3to4`](https://huggingface.co/datasets/Jiayi-Pan/Countdown-Tasks-3to4)

Model: [`Qwen/Qwen2.5-3B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-3B-Instruct)

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load dataset from HuggingFace Hub
dataset_id = 'Jiayi-Pan/Countdown-Tasks-3to4'
dataset = load_dataset(dataset_id, split='train')
# select a random subset of 50k samples
dataset = dataset.shuffle(seed=111).select(range(50_000))

In [ ]:
# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-3B-Instruct')

In [ ]:
# Generate R1 prompt with a prefix for the model to already start with the thinking process
def generate_r1_prompt(numbers, target):
    r1_prefix = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant. You first think about the reasoning process in the mind and then provides the user with the answer.'
        },
        {
            'role': 'user',
            'content': f"""Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can
             only be used once. Show your work in <think> </think> tags. And return the final equation and answer in <answer> </answer> tags, for example,
             <answer> (1 + 2) / 3 = 1 </answer>.
            """
        },
        {
            'role': 'assistant',
            'content': 'Let me solve this step by step.\n<think>'
        }
    ]

    return {
        'prompt': tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True),
        'target': target
    }

In [ ]:
# Convert dataset to the R1 prompt
dataset = dataset.map(lambda x: generate_r1_prompt(x['nums'], x['target']))

# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

## Train the model using GRPO

TRL supports Group Relative Policy Optimization (GRPO) through a dedicated `GRPOTrainer` for aligning LLMs from preference data. The `GRPOTrainer` is a subclass of the `Trainer` from the `transformers` library and supports all the same features, including logging, checkpointing, distributed training, and parameter efficient fine-tuning (PEFT).

The `GRPOTrainer` supports generic **Outcome Reward Models (ORM)** and custom reward functions, that can be used to implement **Rule-Based Reward Models**.

In the Deepseek R1 paper, they implemented Rule-Based Reward Models to verify the correctness of the generated solutions. In this example, we will create two reward functions that:
1. *Format Reward* checks if the generated format is correct `<think> [thinking] </think><answer> [answer] </answer>`
2. *Accuracy Reward* extracts the equation from the `<answer>` tag and evaluates it against the target and if every number is used once.

NOTE: Correct `<answer>` in our example includes the equation, for example, `<answer> 55 + 36 - 7 - 19 </answer>`

In [ ]:
import re

def format_reward_func(completions, target, **kwargs):
    """
    Format: <think>...</think><answer>...</answer>
    Args:
        completions (list[str]): Generated outputs
        target (list[str]): Expected answers

    Returns:
        list[float]: Reward scores
    """
    rewards = []

    for completion, gt in zip(completions, target):
        try:
            # add synthetic <think> as it is already part of the prompt and prefilled for the assistant to more easily match the regex
            completion = "<think>" + completion
            # check if the format is correct
            regex = r"^<think>([^<]*(?:<(?!/?think>)[^<]*)*)<\/think>\n<answer>([\s\S]*?)<\/answer>$"

            _match = re.search(regex, completion, re.DOTALL)
            # if the format is not correct, reward is 0
            if _match is None or len(_match.groups()) != 2:
                rewards.append(0.)
            else:
                rewards.append(1.)

        except Exception:
            rewards.append(0.)

    return rewards


def equation_reward_func(completions, target, nums, **kwargs):
    """
    Evaluates completions based on:
    Mathematical correctness of the answer

    Args:
        completions (list[str]): Generated outputs
        target (list[str]): Expected answers
        nums (list[str]): Numbers used in the equation

    Returns:
        list[float]: Reward scores
    """
    rewards = []

    for completion, gt, numbers in zip(completions, target, nums):
        try:
            # add synthetic <think> as it is already part of the prompt and prefilled for the assistant to more easily match the regex
            completion = "<think>" + completion
            # check if the format is correct
            _match = re.search(r"<answer>(.*?)<\/answer>", completion)
            if _match is None:
                rewards.append(0.)
                continue

            # Extract the answer part from the completion
            equation = _match.group(1).strip()
            # Extract all numbers from the equation
            used_numbers = [int(n) for n in re.findall(r'\d+', equation)]

            # Check if all numbers are used exactly once
            if sorted(used_numbers) != sorted(numbers):
                rewards.append(0.)
                continue

            # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
            allowed_pattern = r'^[\d+\-*/().\s]+$'
            if not re.match(allowed_pattern, equation):
                rewards.append(0.)
                continue

            # Evaluate the equation with restricted globals and locals
            result = eval(equation, {"__builti'ns__": None}, {})
            # Check if the equation is correct and matches the ground truth
            if abs(float(result) - float(gt)) < 1e-5:
                rewards.append(1.)
            else:
                rewards.append(0.)

        except Exception:
            # If evaluation fails, reward is 0
            rewards.append(0.)

    return rewards

We can test our reward functions. Note that none of the examples starts with `<think>` tag as we added it synthetically to the prompt.

In [ ]:
correct_sample_1 = """We need to find an equation using the numbers 19, 36, 55, and 7
exactly once, with basic arithmetic operations, that equals 65. One possible
combination is 55 + 36 - 19 + 7... </think>
<answer> 55 + 36 - 7 - 19 </answer>"""

correct_sample_2 = """ ... </think>
<answer> 55 + 36 - 7 - 19 </answer>"""

wrong_format = """User: Using the numbers [19, 36, 55, 7], create an equation that equals 65."""

wrong_format_2 = """To find the equation that equals 79 using the numbers 95, 78, 6, 88, I'll start by adding 88 and 95:
95 + 88 = 183
Now, let's subtract 104 from 183 to get 79:
183 - 104 = 79
<think> 183 - 104 = 79 </think><think> 183 - 104 = 79 </think><answer> 183 - 104 = 79 </answer>"""

wrong_result = """ ... </think>
<answer> 55 + 36 - 7 - 18 </answer>"""

In [ ]:
test_rewards = format_reward_func(
    completions=[correct_sample_1, correct_sample_2, wrong_format, wrong_format_2, wrong_result],
    target=["65", "65", "65", "65", "65"],
    nums=[[19, 36, 55, 7] * 5]
)
assert test_rewards == [1., 1., 0., 0., 1.], "Reward function is not working"

In [ ]:
test_rewards = equation_reward_func(
    completions=[correct_sample_1, correct_sample_2, wrong_format, wrong_format_2, wrong_result],
    target=["65", "65", "65", "65", "65"],
    nums=[[19, 36, 55, 7]] * 5
)
assert test_rewards == [1.0, 1.0, 0.0, 0.0, 0.0], "Reward function is not working"

Now we can define our remaining training parameters, create a trainer and start training.

In [ ]:
from trl import GRPOConfig, GRPOTrainer, get_peft_config, ModelConfig

# The model we will use as policy
model_config = ModelConfig(
    model_name_or_path='Qwen/Qwen2.5-3B-Instruct',
    torch_dtype='bfloat16',
    attn_implementation='flash_attention_2',
    use_peft=True,
    load_in_4bit=True,
)

# Hyperparameters in the Configuration
training_args = GRPOConfig(
    output_dir='qwen-r1-aha-moment',
    learning_rate=5e-7,
    lr_scheduler_type='cosine',
    logging_steps=10,
    max_steps=100,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    bf16=True,
    # GRPO specific parameters
    max_prompt_length=256,
    max_completion_length=1024, # max length of the generated output for our solution
    num_generations=2,
    beta=0.001,
)

# Trainer setup
trainer = GRPOTrainer(
    model=model_config.model_name_or_path,
    reward_funcs=[format_reward_func, equation_reward_func],
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=get_peft_config(model_config)
)

Now we can start our training:

In [ ]:
trainer.train()
# save model
trainer.save_model(training_args.output_dir)

**Reinforcement Training** is very slow and compute intensive. Running a single step on 1xL4 with Q-LoRA, batch size of 1 only 2 generations per samples takes >20 minutes.

## Distributed training example for GRPO using DeepSpeed and vLLM

TRL added support for distributed training with DeepSpeed and using vLLM for faster generation. We can use the [run_r1_grpo.py](https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/scripts/run_r1_grpo.py) script and the correspodning [receipe config](https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/receipes/grpo-qwen-2.5-3b-deepseek-r1-countdown.yaml) to run the distributed training.

To run with `accelerate`:
```bash
accelerate launch --num_processes 3 --config_file configs/accelerate_configs/deepspeed_zero3.yaml scripts/run_r1_grpo.py --config receipes/grpo-qwen-2.5-3b-deepseek-r1-countdown.yaml

```

The `num_processes` is set to the number of GPUs we have. Make sure to leave the last one for vLLM generation. If we have a Node with 8 A100 GPUs, we need to change the `vllm_device` in the config file to the last index GPU, e.g., we need to set `vllm_device=7` and we set `num_processes` to 7.

## Results

The script saves random completions to the `completion_samples` folder, which you can use to inspect the model's progress. It includes `completion_samples.txt` and `success_completion_samples.txt`. The `completion_samples.txt` includes all completions, while the `success_completion_samples.txt` which correctly solves the equation.

Training Observations:
* At ~50 steps the model has learned the correct format `<think>...</think>\n<answer>...</answer>`.
* At 100 steps the success rate for solving the equation is around 25%. The model starts to "reason" with words see examples below.
* At 200 steps the performance seems to converge much slower and we are at ~40% success rate. The model starts to learn a new "format" where it solves the equation similar to how you would do it programmatically, by trying different combinations and reviewing the results, see "Successfull Reasoning Samples between step 200 and 450".
* At 450 steps we have 50% success rate for solving the equation. The performance still improves slowly and the model kept its new format form from step 200.